In [1]:
import shutil
import os

# Define source and destination paths
src = "/kaggle/input/pycatan"
dst = "/kaggle/working/PyCatan"

# Create destination directory if it doesn't exist
os.makedirs(dst, exist_ok=True)

# Copy all files and subdirectories
shutil.copytree(src, dst, dirs_exist_ok=True)  # Python 3.8+ needed for `dirs_exist_ok`

# Verify
print("Copied files:", os.listdir(dst))

Copied files: ['Images', 'Agents', 'README.md', 'Tests', '.gitignore', 'assets', 'Managers', 'Classes', 'TraceLoader', 'Visualizer', 'Interfaces', 'main.py', 'notebooks']


In [2]:
import shutil
import os

source_dir = "/kaggle/input/agents-haia"
destination_dir = "/kaggle/working/PyCatan/Agents"


os.makedirs(destination_dir, exist_ok=True)


for filename in os.listdir(source_dir):
    source_file = os.path.join(source_dir, filename)
    destination_file = os.path.join(destination_dir, filename)
    
    if os.path.isfile(source_file):
        shutil.copy2(source_file, destination_file)

print(f"✅ All files copied from {source_dir} to {destination_dir}")

import os

os.chdir("/kaggle/working/PyCatan")
print("Current Working Directory:", os.getcwd())



✅ All files copied from /kaggle/input/agents-haia to /kaggle/working/PyCatan/Agents
Current Working Directory: /kaggle/working/PyCatan


In [3]:
import sys

sys.path.append("/kaggle/input/pycatan/Managers")
sys.path.append("/kaggle/input/pycatan/Classes")

# ✅ Import agents
from Agents.RandomAgent import RandomAgent as ra
from Agents.AdrianHerasAgent import AdrianHerasAgent as aha
from Agents.AlexPastorAgent import AlexPastorAgent as apa
from Agents.AlexPelochoJaimeAgent import AlexPelochoJaimeAgent as apja
from Agents.CarlesZaidaAgent import CarlesZaidaAgent as cza
from Agents.CrabisaAgent import CrabisaAgent as ca
from Agents.EdoAgent import EdoAgent as ea
from Agents.PabloAleixAlexAgent import PabloAleixAlexAgent as paaa
from Agents.SigmaAgent import SigmaAgent as sa
from Agents.TristanAgent import TristanAgent as ta

# ✅ Import GameDirector
from Managers.GameDirector import GameDirector

# ✅ Confirm successful imports
print("All agents and GameDirector imported successfully!")



All agents and GameDirector imported successfully!


In [4]:
import random
import numpy as np
import json
import multiprocessing
from deap import base, creator, tools
from math import exp
import time

AGENTS = [ra, aha, apa, apja, cza, ca, ea, paaa, sa, ta]

# Parámetros de ejecución
# ============================================================================
PARAMS = {
    "popu_size": 100,             # Tamaño de la población inicial
    "generaciones": 50,          # Cantidad de generaciones
    "prob_cruce": 0.5,            # Probabilidad de cruce
    "prob_mutacion": 0.5,         # Probabilidad de mutación
    "cantidad_partidas": 50,     # Partidas para evaluar el fitness
    "oponentes_competitivos": False,
    "desordenar": False,
    "elitism_size": 35,
    "roulette_size": 35,
    "k_agentes_mutados": 3,
}


def softmax(vector):
    exp_vals = np.exp(vector - np.max(vector))  # Evitar overflow numérico
    return list(exp_vals / np.sum(exp_vals))

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()


# Individuos
# ============================================================================
def initIndividual():
    return creator.Individual(np.random.dirichlet(np.ones(len(AGENTS))).tolist())

toolbox.register("individual", initIndividual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


# Cruce uniforme (uniform_crossover)
# ============================================================================
def uniform_crossover(ind1, ind2):
    child1 = [ind1[i] if random.random() < 0.5 else ind2[i] for i in range(len(ind1))]
    child2 = [ind2[i] if random.random() < 0.5 else ind1[i] for i in range(len(ind1))]
    child1[:] = softmax(child1)
    child2[:] = softmax(child2)
    return creator.Individual(child1), creator.Individual(child2)

toolbox.register("mate", uniform_crossover)


# Mutación (mutation_among_top_k)
# ============================================================================
def mutation_among_top_k(individual, k=3, indpb=0.5):

    sorted_indices = sorted(range(len(individual)), key=lambda i: individual[i], reverse=True)
    top_k = sorted_indices[:k]
    if random.random() < indpb:
        i1, i2 = random.sample(top_k, 2)
        individual[i1], individual[i2] = individual[i2], individual[i1]
    perturb = [random.gauss(0, 0.01) for _ in individual]
    individual = [max(x + dx, 0) for x, dx in zip(individual, perturb)]
    individual[:] = softmax(individual)
    
    return (creator.Individual(individual),)

toolbox.register("mutate", mutation_among_top_k,
                 k=PARAMS["k_agentes_mutados"],
                 indpb=0.5)

#Evaluación
# ============================================================================
def evaluate_individual(individual, params):
    my_index = np.argmax(individual)
    my_agent = AGENTS[my_index]
    score_total = 0.0
    valid_games = 0
    
    if params["oponentes_competitivos"]:
        sorted_indices = sorted(range(len(AGENTS)), key=lambda i: individual[i], reverse=True)
        opponents = [AGENTS[idx] for idx in sorted_indices if idx != my_index][:3]
    else:
        available_opponents = [agent for agent in AGENTS if agent != my_agent]
    
    for _ in range(params["cantidad_partidas"]):
        # Si no son fijos los oponentes, elegimos 3 al azar
        if not params["oponentes_competitivos"]:
            opponents = random.sample(available_opponents, min(3, len(available_opponents)))
        
        game_players = [my_agent] + opponents
        
        # Desordenar el orden de los jugadores
        if params["desordenar"]:
            random.shuffle(game_players)
        
        my_position = game_players.index(my_agent)
        
        try:
            game_director = GameDirector(agents=game_players, max_rounds=200, store_trace=False)
            game_trace = game_director.game_start(print_outcome=False)
            
            last_round = max(game_trace["game"].keys(), key=lambda r: int(r.split("_")[-1]))
            last_turn = max(game_trace["game"][last_round].keys(),
                            key=lambda t: int(t.split("_")[-1].lstrip("P")))
            vp = game_trace["game"][last_round][last_turn]["end_turn"]["victory_points"]
            
            # Puntajes finales: premiar con (5 - rank), de modo que 1er lugar = +4, 2do lugar=+3, etc.
            sorted_players = sorted(vp.keys(), key=lambda j: int(vp[j]), reverse=True)
            rank = sorted_players.index(f"J{my_position}") + 1
            score_total += (5 - rank)
            valid_games += 1
            
        except Exception as e:
            # Si hay error en la partida, se ignora
            print(f"Game error (ignored): {str(e)}")
    
    avg_score = score_total / valid_games if valid_games > 0 else 0
    return (avg_score,)  

toolbox.register("evaluate", evaluate_individual, params=PARAMS)

# Selección
# ============================================================================
toolbox.register("selectRoulette", tools.selRoulette)
toolbox.register("selectBest", tools.selBest)

# Evolución
# ============================================================================
def run_evolution():
    # 1) Crear la población inicial
    population = toolbox.population(n=PARAMS["popu_size"])
    
    # 2) Evaluar la población inicial
    print("Evaluando población inicial")
    fitnesses = list(toolbox.map(toolbox.evaluate, population))
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit  # fit es una tupla (avg_score, )
    
    # Estadísticas
    stats = tools.Statistics(lambda ind: ind.fitness.values[0])
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)
    
    logbook = tools.Logbook()
    logbook.header = ["gen", "nevals"] + stats.fields
    best = None
    
    # 3) Comienza el ciclo de generaciones
    for gen in range(PARAMS["generaciones"]):
        start_time = time.time()
        
        try:
            # Selección con elitismo
            elite = list(map(toolbox.clone, toolbox.selectBest(population, PARAMS["elitism_size"])))
            # Selección con ruleta
            offspring = list(map(toolbox.clone, toolbox.selectRoulette(population, PARAMS["roulette_size"])))
            
            # Remplazo nueva pob y evaluación ind nuevos (exploracion)
            new_population = toolbox.population(n=len(population) - PARAMS["elitism_size"] - PARAMS["roulette_size"])
            print("Evaluando población nueva")
            fitnesses_newpop = list(toolbox.map(toolbox.evaluate, new_population))
            for ind, fit in zip(new_population, fitnesses_newpop):
                ind.fitness.values = fit  
           
        except Exception as e:
            print(f"Selection error: {str(e)}")
            continue
        
        # Cruce
        for i in range(1, len(offspring)):
            if (random.random() < PARAMS["prob_cruce"]):
                try:
                    parent1 = toolbox.clone(offspring[i-1])
                    parent2 = toolbox.clone(offspring[i])
                    child1, child2 = toolbox.mate(parent1, parent2)
                    offspring[i-1] = child1
                    offspring[i] = child2
                    offspring[i].fitness.values = toolbox.evaluate(offspring[i])
                    offspring[i-1].fitness.values = toolbox.evaluate(offspring[i-1])
                    
                except Exception as e:
                    print(f"Error in crossover at pair {i-1}-{i}: {str(e)}")
                    # Si falla el cruce, se conservan los originales
                    offspring[i-1] = toolbox.clone(offspring[i-1])
                    offspring[i] = toolbox.clone(offspring[i])
                    continue
        
        # Mutación
            if random.random() < PARAMS["prob_mutacion"]:
                try:
                    mutant = toolbox.clone(offspring[i])
                    mutant, = toolbox.mutate(mutant)
                    offspring[i] = mutant
                    offspring[i].fitness.values = toolbox.evaluate(offspring[i])
                except Exception as e:
                    print(f"Error mutating individual {i}: {str(e)}")
                    offspring[i] = toolbox.clone(offspring[i])
        
        # Reemplazar población con nueva generación
        offspring.extend(new_population)
        offspring.extend(elite)
        population[:] = offspring
        
        # 4) Actualizar el mejor individuo global
        current_best = tools.selBest(population, 1)[0]
        if best is None or current_best.fitness.values[0] > best.fitness.values[0]:
            best = toolbox.clone(current_best)
        
        # 5) Registrar estadísticas
        record = stats.compile(population)
        logbook.record(gen=gen, nevals=len(population), **record)
        print(logbook.stream)
        
        gen_time = time.time() - start_time
        print(f"Generación {gen} completada en {gen_time/60:.2f} minutos")

            # Create structured data for this generation
        gen_data = {
            "generation": gen,
            "stats": {
                "avg": record["avg"],
                "min": record["min"],
                "max": record["max"]
            },
            "population_size": len(population),
            "best_individual": {
                "fitness": float(tools.selBest(population, 1)[0].fitness.values[0]),
                "agent": str(AGENTS[np.argmax(tools.selBest(population, 1)[0])])
            }
        }
        
        output_filename = "00_evolucion_j0_opRand_100part_20gen_70ind.json"
        try:
            with open(output_filename, 'w') as f:
                json.dump(gen_data, f, indent=2)
        except Exception as e:
            print(f"Error saving progress: {str(e)}")
       
      
    return population, logbook, best

# ============================================================================
def main():
    # Ejecutar la evolución
    final_pop, log, best = run_evolution()
    
    # Guardar resultados en archivo JSON
    results = {
        "parameters": PARAMS,
        "best_individual": best,
        "logbook": log,
        "best_agent": str(AGENTS[np.argmax(best)]),
        "final_population_fitness": [ind.fitness.values[0] for ind in final_pop]
    }
    
    with open("00_j0_opRand_100part_20gen_70ind.json", "w") as f:
        json.dump(results, f, indent=4)
    
    # Imprimir resultados finales
    print(f"\nMejor individuo: {best}")
    print(f"Mejor fitness: {best.fitness.values[0]:.3f}")
    print(f"Agente seleccionado: {AGENTS[np.argmax(best)]}")

# ============================================================================
if __name__ == "__main__":
    # Uso de multiprocessing para evaluar en paralelo
    pool = multiprocessing.Pool(processes=4)
    toolbox.register("map", pool.map)
    
    try:
        main()
    finally:
        pool.close()
        pool.join()




Evaluando población inicial
Evaluando población nueva
gen	nevals	avg   	min 	max 
0  	100   	3.2072	2.28	3.58
Generación 0 completada en 13.32 minutos
Evaluando población nueva
1  	100   	3.253 	2.34	3.68
Generación 1 completada en 13.05 minutos
Evaluando población nueva
Game error (ignored): No hay nodos válidos disponibles para iniciar el juego
2  	100   	3.26087	2.3 	3.68
Generación 2 completada en 11.68 minutos
Evaluando población nueva
3  	100   	3.2022 	2.26	3.68
Generación 3 completada en 14.48 minutos
Evaluando población nueva
Game error (ignored): No hay nodos válidos disponibles para iniciar el juego
4  	100   	3.21189	2.2 	3.68
Generación 4 completada en 15.89 minutos
Evaluando población nueva
5  	100   	3.2584 	2.34	3.68
Generación 5 completada en 16.01 minutos
Evaluando población nueva
6  	100   	3.2358 	2.22	3.68
Generación 6 completada en 14.77 minutos
Evaluando población nueva
7  	100   	3.2406 	2.24	3.68
Generación 7 completada en 14.73 minutos
Evaluando población nuev